# Simple Moving Average (Long only)

We determine the 12-period simple moving average (referred to as 'SMA12') and compare it with the price at that time. We (subjectively) select 12 since SMA12 would be the average price over one hour.

There are two conditions which we check.

1. If the price is greater than the SMA12, we go long. We continue to stay invested until the square-off condition is satisfied.

2. When the price becomes less than the SMA12, we square off our long position.

Our trading rules can be stated as

1. Buy when price > SMA12
2. Square off when price < SMA12

Import the required libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import warnings 

warnings.filterwarnings('ignore')

Fix the backtesting period

In [2]:
# Backtesting period should be last 55 days data in 5 minute level 
end = pd.datetime.now().date()
start = end - pd.Timedelta(days=55)

Create a class and create instance attributes/methods which are required for backtesting

In [3]:
# Create a class
class backtesting_SMA:
    
    # Create Instance attribute and methods
    def __init__(self,ticker,start,end,sma_range):
        
        self.ticker = ticker
        self.start = start
        self.end = end
        self.sma_range = sma_range
        
        self.fetch_data()
        self.indicators()
        self.signals()
        self.positions()
        self.returns()
        
    #To fetch the required data from yahoo finance    
    def fetch_data(self):
        self.df = yf.download(self.ticker,self.start,self.end,interval='5m')
    
    # To compute the SMA indicator
    def indicators(self):
        self.df['sma'] = self.df['Adj Close'].rolling(window=self.sma_range).mean()
    
    # To generate the long only signal
    def signals(self):
        self.df['signals'] = np.where(self.df['Adj Close'] > self.df['sma'],1,0)
    
    # To mark the positions taken
    def positions(self):
        self.df['positions'] = self.df['signals'].shift()
    
    #To compute the returns generated
    def returns(self):
        self.df['BnH Returns'] = self.df['Adj Close'].pct_change()
        self.df['Strategy Returns'] = self.df['positions'] * self.df['BnH Returns']
        # BnH returns is the returns generated if bought and hold it for the mentioned period simply without any strategy
        self.df['BnH Returns'] = 1+self.df['BnH Returns']
        # Strategy Returns is the returns generated due to implementation of this strategy on this historical data
        self.df['Strategy Returns'] = 1+self.df['Strategy Returns']
        Strat_Returns = self.df['Strategy Returns'].cumprod()[-1]
        BnH_Returns = self.df['BnH Returns'].cumprod()[-1]
        print('Buy and Hold Returns :', self.df['BnH Returns'].cumprod()[-1])
        print('Strategy Returns :', self.df['Strategy Returns'].cumprod()[-1])
        return Strat_Returns, BnH_Returns
    

In [4]:
# Try with Nifty index by passing SMA range as 12, this will return the buy and hold return & strategy return 
Nifty_backtesting_SMA = backtesting_SMA('^NSEI',start,end,12)

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.0000496582804133
Strategy Returns : 1.03392068873173


Optimise the strategy by considering few other indices with the various short term moving average range 

In [5]:
sma_list = []
Returns=[]
indices=[]
indice_list=['^NSEI','SPY','^CNXIT','^NSEBANK','^GSPC']
for i in range(8,15,1):
    for j in indice_list:
        a = backtesting_SMA(j,start,end,i)
        sma_list.append(i)
        Returns.append(a.returns())
        indices.append(j)
Strat = [x[0] for x in Returns]
BnH = [x[1] for x in Returns]
    

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.0000496582804133
Strategy Returns : 1.0350174260983787
Buy and Hold Returns : 1.0000496582804133
Strategy Returns : 1.0350174260983787
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.9534698226109998
Strategy Returns : 1.0291835316472961
Buy and Hold Returns : 0.9534698226109998
Strategy Returns : 1.0291835316472961
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.9391186837817751
Strategy Returns : 0.9972963065081235
Buy and Hold Returns : 0.9391186837817751
Strategy Returns : 0.9972963065081235
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.057767070269866
Strategy Returns : 1.03628157234412
Buy and Hold Returns : 1.057767070269866
Strategy Returns : 1.03628157234412
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns 

In [6]:
# Create a dataframe and validate the returns generated for various indices with different SMA range
Results = pd.DataFrame({'Indices':indices,'SMA':sma_list,'Strategy Returns':Strat,'BnH Returns':BnH})


In [7]:
# Shows the record in descending order which has maximum return 
Results.sort_values(by='Strategy Returns',inplace=True,ascending=False)

Note : This is simple strategy to learn how to backtest and optimize the parameters by reusing the code.

In [8]:
Results.head(20)

,Indices,SMA,Strategy Returns,BnH Returns
33,^NSEBANK,14,1.068326,1.057767
28,^NSEBANK,13,1.063250,1.057767
8,^NSEBANK,9,1.061614,1.057767
13,^NSEBANK,10,1.060758,1.057767
23,^NSEBANK,12,1.050544,1.057767
5,^NSEI,9,1.044396,1.000050
18,^NSEBANK,11,1.043959,1.057767
4,^GSPC,8,1.038184,0.955254
3,^NSEBANK,8,1.036282,1.057767
0,^NSEI,8,1.035017,1.000050
